In [62]:
%pip install pandas seaborn matplotlib scipy statsmodels

Note: you may need to restart the kernel to use updated packages.


In [63]:
import pandas as pd 
import seaborn as sb 
import matplotlib.pyplot as mp 
import scipy 
import numpy as np
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [64]:
# leitura do arquivo
tabela = pd.read_csv('TabelaSalarial.csv')
tabela

,N,Estado Civil,Grau de Instrução,N de Filhos,Salario (x Sal Min),Idade (em anos),Região de Procedência
0,1,solteiro,ensino fundamental,NaN,4.00,26,interior
1,2,casado,ensino fundamental,1.0,4.56,32,capital
2,3,casado,ensino fundamental,2.0,5.25,36,capital
3,4,solteiro,ensino médio,NaN,5.73,20,outra
4,5,solteiro,ensino fundamental,NaN,6.26,40,outra
5,6,casado,ensino fundamental,0.0,6.66,28,interior
6,7,solteiro,ensino fundamental,NaN,6.86,41,interior
7,8,solteiro,ensino fundamental,NaN,7.39,43,capital
8,9,casado,ensino médio,1.0,7.59,34,capital
9,10,solteiro,ensino médio,NaN,7.44,23,outra


In [65]:

# Entender o arquivo para manipular dados inconsistntes
tabela.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   N                      36 non-null     int64  
 1   Estado Civil           36 non-null     object 
 2   Grau de Instrução      36 non-null     object 
 3   N de Filhos            20 non-null     float64
 4   Salario (x Sal Min)    36 non-null     float64
 5   Idade (em anos)        36 non-null     int64  
 6   Região de Procedência  36 non-null     object 
dtypes: float64(2), int64(2), object(3)
memory usage: 2.1+ KB


In [66]:
# Como os dados estão bem estruturados, vamos só preencher as células NaN com zero
tabela_clean = tabela.fillna(0)
tabela_clean.describe()

,N,N de Filhos,Salario (x Sal Min),Idade (em anos)
count,36.000000,36.000000,36.000000,36.000000
mean,18.500000,0.916667,11.122222,34.583333
std,10.535654,1.250714,4.587458,6.737422
min,1.000000,0.000000,4.000000,20.000000
25%,9.750000,0.000000,7.552500,30.000000
50%,18.500000,0.000000,10.165000,34.500000
75%,27.250000,2.000000,14.060000,40.000000
max,36.000000,5.000000,23.300000,48.000000


In [67]:
# Quais são os fatores que mais influenciam no salário?

tabela_numerica1 = tabela_clean.select_dtypes(include=[np.number])
tabela_numerica = tabela_numerica1.drop(columns=['N'])



In [68]:

pearson_correl = tabela_numerica.corr(method='pearson')
display(pearson_correl)

,N de Filhos,Salario (x Sal Min),Idade (em anos)
N de Filhos,1.000000,0.375551,0.422981
Salario (x Sal Min),0.375551,1.000000,0.363362
Idade (em anos),0.422981,0.363362,1.000000


In [ ]:
# Interpretação dos valores de R^2:
#
#  0.00 a 0.02  : Muito Fraca
#  0.02 a 0.10  : Fraca
#  0.10 a 0.30  : Moderada
#  0.30 a 0.50  : Forte
#  0.5  a 1.00  : Muito Forte (Verifique os dados!)
#
# Com base na tabela de Pearson todas variáveis apresentam uma correlação forte entre i

In [74]:
# Convertendo variáveis qualitativas para variáveis dummy
tabela_clean_dummies = pd.get_dummies(tabela_clean, columns=['Grau de Instrução', 'Região de Procedência'], drop_first=True)
tabela_clean_dummies['Estado Civil'] = tabela_clean_dummies['Estado Civil'].map({'solteiro': 0, 'casado': 1})

# Renomear colunas para remover espaços e caracteres especiais
tabela_clean_dummies.columns = tabela_clean_dummies.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Variáveis numéricas
numeric_vars = ['Idade_em_anos', 'Salario_x_Sal_Min', 'N_de_Filhos']

# Variáveis qualitativas dummy
qualitative_vars = [col for col in tabela_clean_dummies.columns if col not in numeric_vars]

# Função para calcular o R^2
def calculate_r_squared(dependent_var, independent_vars):
    formula = f"{dependent_var} ~ " + " + ".join(independent_vars)
    model = ols(formula, data=tabela_clean_dummies).fit()
    return model.rsquared

# Criar a tabela de R^2
r_squared_table = pd.DataFrame(index=numeric_vars, columns=qualitative_vars)

for numeric_var in numeric_vars:
    for qual_var in qualitative_vars:
        r_squared = calculate_r_squared(numeric_var, [qual_var])
        r_squared_table.loc[numeric_var, qual_var] = r_squared

display(r_squared_table)

,N,Estado_Civil,Grau_de_Instrução_ensino_médio,Grau_de_Instrução_superior,Região_de_Procedência_interior,Região_de_Procedência_outra
Idade_em_anos,0.112015,0.009095,0.029391,0.007081,0.020142,0.000013
Salario_x_Sal_Min,0.945531,0.06125,0.008061,0.280078,0.004472,0.012655
N_de_Filhos,0.121109,0.442009,0.012684,0.000913,0.009132,0.033735


In [ ]:
# Interpretação dos valores de R^2
#
#  0   a 0.1   : Muito Fraca
#  0.1 a 0.3   : Fraca
#  0.3 a 0.7   : Moderada
#  0.7 a 1.0   : Forte 
#
# 
